In [ ]:
!pip install wfdb

In [ ]:
#!wget -r -N -c -np https://physionet.org/files/ptb-xl/1.0.3/

In [2]:
PATH = './data/'
SAMPLING_RATE = 100

In [3]:
import pandas as pd
import numpy as np
import wfdb
import ast
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [4]:
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

path = PATH
sampling_rate=SAMPLING_RATE

# load and convert annotation data
Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

# Load raw signal data
#X = load_raw_data(Y, sampling_rate, path)

# Load scp_statements.csv for diagnostic aggregation
agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

# Apply diagnostic superclass
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Split data into train and test
#test_fold = 10
# Train
#X_train = X[np.where(Y.strat_fold != test_fold)]
#y_train = Y[(Y.strat_fold != test_fold)].diagnostic_superclass
# Test
#X_test = X[np.where(Y.strat_fold == test_fold)]
#y_test = Y[Y.strat_fold == test_fold].diagnostic_superclass

In [8]:
class PTBDataset(Dataset):

    def __init__(self, df, path, sampling_rate):
        self.df = df
        self.path = path
        self.sampling_rate = sampling_rate
        if sampling_rate == 100:
            self.data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
        else:
            self.data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
        self.data = np.array([signal for signal, meta in self.data])
        print(self.data[0])
        

    def __getitem__(self, idx):
        signal = torch.FloatTensor(self.data[idx])                 
        return signal

    def __len__(self):
        return len(self.df)

In [9]:
dataset = PTBDataset(Y, path, 100)
    

[[-0.119 -0.055  0.064 ... -0.026 -0.039 -0.079]
 [-0.116 -0.051  0.065 ... -0.031 -0.034 -0.074]
 [-0.12  -0.044  0.076 ... -0.028 -0.029 -0.069]
 ...
 [ 0.069  0.    -0.069 ...  0.024 -0.041 -0.058]
 [ 0.086  0.004 -0.081 ...  0.242 -0.046 -0.098]
 [ 0.022 -0.031 -0.054 ...  0.143 -0.035 -0.12 ]]


In [10]:
print(len(dataset))
x = dataset[0]
x.shape

21799


torch.Size([1000, 12])

In [11]:
dataloader = DataLoader(dataset=dataset, batch_size=128, num_workers=0)
